In [39]:
import tensorflow_hub as hub
import tensorflow as tf 
import tensorflow_text
import keras
import tensorflow_addons as tfa

In [5]:
# Set random seed for reproducibility
tf.keras.utils.set_random_seed(42)

In [49]:
def rnn_bert(n_classes = 3, layers_shape = (32, 32), rnn_cell = 'lstm', learning_rate=3e-5, label_smoothing=0.1):

    # URLS to BERT encoder and preprocess models on tfhub
    bert_model_url = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
    bert_preprocess_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

    # Model will take in whole text sequences
    text_input = tf.keras.Input(shape=(), dtype=tf.string, name='text')

    # Preprocess the input for BERT encoder
    preprocess_layer = hub.KerasLayer(bert_preprocess_url, name='preprocess')

    encoder_input = preprocess_layer(text_input)

    # Encode the input
    bert_encoder = hub.KerasLayer(bert_model_url, name='bert_encoder')
    bert_output = bert_encoder(encoder_input)
    
    sequence = bert_output['sequence_output']   
    # Add RNN layers
    last_layer = sequence
    for i in range(len(layers_shape)):
        # Run the sequnece through the RNN layer
        ret_seq = True
        if i == len(layers_shape) - 1:
            ret_seq = False

        if rnn_cell == 'lstm':
            new_layer = tf.keras.layers.LSTM(layers_shape[i], return_sequences=ret_seq)(last_layer)

        elif rnn_cell == 'gru':
            new_layer = tf.keras.layers.Bidirectional(
                tf.keras.layers.GRU(layers_shape[i], return_sequences=ret_seq),
                merge_mode='sum')(last_layer)

        last_layer = new_layer

    # Predict the resulting classes with a softmax
    classes_pred = tf.keras.layers.Dense(n_classes, activation='softmax')(last_layer)

    # Set metrics for accuracy
    metrics= [
        tf.keras.metrics.CategoricalAccuracy(name='accuracy'), 
        tfa.metrics.F1Score(n_classes, average='weighted', name='f1_weighted'),
        tfa.metrics.F1Score(n_classes, average='macro', name='f1_macro'),
        tfa.metrics.FBetaScore(n_classes, average='macro', beta=0.5, name='f_0.5 macro')
        ]
  
    # Finalize model
    model = keras.Model(inputs=text_input, outputs=classes_pred)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), 
        loss=tf.losses.CategoricalCrossentropy(label_smoothing=label_smoothing), 
        metrics=metrics
    )

    print(model.summary())

    return model

In [34]:
import re
import numpy as np

def preprocess(data : np.ndarray):
    """
    Text Preprocessor 
 
    Preprocess the text, replacing links with '[link]' and other
    special character removals.

    :param data: Source Dataframe
    :return: Preprocessed text
    """

    # Initial text cleanup
    for i in range(data.shape[0]):
        post = data[i]
        post = re.sub('\n', ' ', post) # remove newline chars
        post = re.sub('[\(\)\[\]\{\}]', ' ', post) # remove braces
        post = post.lower()
        data.iloc[i] = re.sub('http[\w:/\.\=\#\?\-\$\&]+', '[link]', post)

    return data

In [53]:
from sklearn.model_selection import train_test_split, StratifiedKFold

def create_dataset(data, idxs):
    train_data, val_data, train_target, val_target = train_test_split(data.text[idxs], data.target[idxs], test_size=0.2, shuffle=True, random_state=42, stratify=data.target[idxs])
    
    train_ds = tf.data.Dataset.from_tensor_slices((train_data, train_target))
    val_ds = tf.data.Dataset.from_tensor_slices((val_data, val_target))

    return train_ds, val_ds

def train_model(data):
    data.text = preprocess(data.text)
    n_classes = len(data.target.unique())
    
    folder = StratifiedKFold()
    tf.debugging.disable_traceback_filtering()
    for train_idxs, test_idxs in folder.split(data.text, data.target):
        train_ds, val_ds = create_dataset(data, train_idxs)
        model = rnn_bert(n_classes)
        model.fit(train_ds, validation_data = val_ds, epochs=100)

        # Calculate metrics
        loss, accuracy, f1_macro, f1_weighted, f05_macro = model.evaluate(data.text[test_idxs], data.target[test_idxs], use_multiprocessing=True)

        print(f'Acc : {accuracy}, F1 : {f1_macro}, F1_weighted : {f1_weighted}, F05 : {f05_macro}')

In [35]:
import pandas as pd

# Load data
data = pd.read_json('..\\Data\\train_dataset_alt_new.json', encoding='utf-8')
data

,text,type,id,target
0,DFV's Roaring Kitty Spreadsheet Recreated with...,reddit_post,qicrti,2
1,From an open-source data engineer... Is stock ...,reddit_post,qia0tt,2
2,DWAC-Warrant / long-dated call arbitrage? Long...,reddit_post,qhvtqi,2
3,"$CHWY DD This DD is short, because the logic i...",reddit_post,qhnq2y,2
4,Time to Hedge Long Equity Exposure In my opini...,reddit_post,qh7yld,2
...,...,...,...,...
1652,#GME #Gamestop still out of stock on PS5 https...,tweet,1375079563844280324,0
1653,#GME #GameStop don't be alarmed. This morning ...,tweet,1466788507494592512,1
1654,"Ironically, GameStop might well stop the whole...",tweet,1354525266547236865,1
1655,@DiamondApe8 @Mindy831518 @jkarn88 @MInk547076...,tweet,1476305947442044930,1


In [54]:
train_model(data)

C:\Users\Samo\AppData\Local\Temp\ipykernel_22192\4061643057.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.iloc[i] = re.sub('http[\w:/\.\=\#\?\-\$\&]+', '[link]', post)


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 preprocess (KerasLayer)        {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                    

ValueError: in user code:

    File "c:\Users\Samo\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\Samo\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Samo\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow\python\distribute\distribute_lib.py", line 1315, in run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    File "c:\Users\Samo\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow\python\distribute\distribute_lib.py", line 2891, in call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    File "c:\Users\Samo\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow\python\distribute\dist